In [7]:
import numpy as np
from scipy.stats import norm
from py_vollib.black_scholes import black_scholes as bs 
from py_vollib.black_scholes.greeks.analytical import delta, gamma, vega, rho, theta

# Black Scholes Formula

$C = \Phi(d_1)S_t - \Phi(d_2)Ke^{-rt}$


$P = \Phi(-d_2)Ke^{-rt} - \Phi(-d_1)S_t$

$d_1 = \frac{ln(\frac{S_t}{K}) + (r + \frac{\sigma^2}{2})t)}{\sigma \sqrt{t}}$

$d_2 = d_1 - \sigma \sqrt{t}$

In [10]:
def black_scholes(r, S, K, T, sigma, type):

    d1 = (np.log(S/K) + (r+ sigma**2/2)*T)/(sigma*np.sqrt(T))

    d2 = d1 - sigma*np.sqrt(T)

    if type == "CALL":
        price = S*norm.cdf(d1,0,1) - K*np.exp(-r*T)*norm.cdf(d2,0,1)
    elif type == "PUT":
        price = K*np.exp(-r*T)*norm.cdf(-d2,0,1) - S*norm.cdf(-d1, 0, 1)

    return price


In [14]:
# confirm our model

r = 0.01    #risk free rate
S = 30 
K = 40
T = 240/365
sigma = 0.3
type = "CALL"
print("our call option price: " + str(black_scholes(r, S, K, T, sigma, type)))
print("correct call option price: " + str(bs("c", S, K, T, r, sigma)))


type = "PUT"
print("our put option price: " + str(black_scholes(r, S, K, T, sigma, type)))
print("correct put option price: " + str(bs("p", S, K, T, r, sigma)))

our call option price: 0.5132843798399405
their call option price: 0.5132843798399411
our put option price: 10.251133491653508
their put option price: 10.2511334916535


# Delta $\Delta$
Delta measures the rate of change of our option value $V$ with respect to the underlying asset: $\Delta = \frac{\partial V}{\partial S}$



$\Delta_{Call}= \frac{\partial (\Phi(d_1)S - \Phi(d_2)Ke^{-rt})}{\partial S}$

$= \Phi (d_1) + S \phi(d_1) \frac{\partial d_1}{\partial S} - e^{-RT}K\phi(d_2) \frac{\partial d_2}{\partial S}$

$= \Phi(d_1) + S \phi(d_1)\frac{\partial d_1}{\partial S} - e^{-rT}K\phi(d_2) \frac{\partial d_2}{\partial S}$

$= \Phi(d_1) + S \phi(d_1) \frac{1}{S \sigma \sqrt{T}} - e^{-rT}K\phi(d_2) \frac{1}{S \sigma \sqrt{T}}$

$= \Phi(d_1) + \frac{\phi (d_1)}{\sigma \sqrt{T}}(1 - e^{(d_1^2 - d_2^2)/2} \frac{K}{S e^{rT}})$

$= \Phi(d_1) + \frac{\phi (d_1)}{\sigma \sqrt{T}}(1 - \frac{S e^{rT}}{K} \frac{K}{S e^{rT}})$

$= \Phi(d_1)$




Put-Call-Parity:

$C + Ke^{-rT} = S + P$ 

$\Leftrightarrow P = C  + Ke^{-rT} - S$

$\Leftrightarrow \frac{\partial P}{\partial S} = \frac{\partial ( C  + Ke^{-rT} - S)}{\partial S}$

$\Leftrightarrow \Delta_{Put} = \Delta_{Call} - 1 = \Phi(d_1) - 1 = - \Phi(-d_1)$



In [22]:
def delta_black_scholes(r, S, K, T, sigma, type):

    d1 = (np.log(S/K) + (r+ sigma**2/2)*T)/(sigma*np.sqrt(T))

    if type == "CALL":
        price = norm.cdf(d1,0,1)
    elif type == "PUT":
        price = -norm.cdf(-d1,0,1)

    return price


type = "CALL"
print("our call option delta: " + str(delta_black_scholes(r, S, K, T, sigma, type)))
print("correct call option delta: " + str(delta("c", S, K, T, r, sigma)))


type = "PUT"
print("our put option delta: " + str(delta_black_scholes(r, S, K, T, sigma, type)))
print("correct put option delta: " + str(delta("p", S, K, T, r, sigma)))

our call option delta: 0.15058613984880015
correct call option delta: 0.15058613984880015
our put option delta: -0.8494138601511998
correct put option delta: -0.8494138601511998


# Gamma $\Gamma$

Describes the rate of change between the delta $\Delta$ of an option and the price of the underlying. A high gamma indicates a high change in the delta when the price of the underlying changes. $\Gamma = \frac{\partial \Delta}{\partial S}$


$\Gamma_{Call} = \frac{\partial (\Phi(d_1))}{\partial S}$

$= \phi(d_1) \cdot \frac{\partial d_1}{\partial S}$

$=  \frac{\phi(d_1)}{S \sigma \sqrt{T}}$

$\Gamma_{Put} = \frac{\partial ( - \Phi( - d_1))}{\partial S}$

$= -\phi(d_1) \cdot -\frac{\partial d_1}{\partial S}$

$=  \frac{\phi(d_1)}{S \sigma \sqrt{T}}$

In [25]:
def gamma_black_scholes(r, S, K, T, sigma, type):

    d1 = (np.log(S/K) + (r+ sigma**2/2)*T)/(sigma*np.sqrt(T))

    if type == "CALL":
        return norm.pdf(d1,0,1)/(S * sigma * np.sqrt(T))
    elif type == "PUT":
        return norm.pdf(d1,0,1)/(S * sigma * np.sqrt(T))



type = "CALL"
print("our call option gamma: " + str(gamma_black_scholes(r, S, K, T, sigma, type)))
print("correct call option gamma: " + str(gamma("c", S, K, T, r, sigma)))


type = "PUT"
print("our put option gamma: " + str(gamma_black_scholes(r, S, K, T, sigma, type)))
print("correct put option gamma: " + str(gamma("p", S, K, T, r, sigma)))

our call option gamma: 0.03203161102008452
correct call option gamma: 0.03203161102008452
our put option gamma: 0.03203161102008452
correct put option gamma: 0.03203161102008452


# Vega $\Lambda$

Describes the rate of change between the option value and volatility. So $\Lambda = \frac{\partial V}{\partial \sigma}$

$\Lambda_{Call}= \frac{\partial (\Phi(d_1)S - \Phi(d_2)Ke^{-rt})}{\partial \sigma}$

$= S\phi(d_1) \frac{\partial d_1}{\partial \sigma} - Ke^{-rt} \phi(d_2) \frac{\partial d_2}{\partial \sigma}$

$= S\phi(d_1) \frac{\partial d_1}{\partial \sigma} - Ke^{-rt} \phi(d_2) \frac{\partial d_2}{\partial \sigma}$

$=S \phi (d_1) \sqrt{t}$

$\Lambda_{Call} = \Lambda_{Put}$

important note: This formula calculates the change in value for a 100% change in the volatility. It is much more practical to have the change in value for a 1% change in volatility which is why I use $\Lambda' = 0.01 \cdot \Lambda$

In [32]:
def vega_black_scholes(r, S, K, T, sigma, type):

    d1 = (np.log(S/K) + (r+ sigma**2/2)*T)/(sigma*np.sqrt(T))

    if type == "CALL" or type == "PUT":
        return 0.01 * S * norm.pdf(d1,0,1) * np.sqrt(T)

type = "CALL"
print("our call option vega: " + str(vega_black_scholes(r, S, K, T, sigma, type)))
print("correct call option vega: " + str(vega("c", S, K, T, r, sigma)))


type = "PUT"
print("our put option vega: " + str(vega_black_scholes(r, S, K, T, sigma, type)))
print("correct put option vega: " + str(vega("p", S, K, T, r, sigma)))

our call option vega: 0.056867079290451414
correct call option vega: 0.05686707929045143
our put option vega: 0.056867079290451414
correct put option vega: 0.05686707929045143


# Theta $\Theta$

Theta measures the rate at which the value of an option decreases due to the passage of time $\Theta = \frac{\partial V}{\partial t}$

$\Theta_{Call} = \frac{\partial (\Phi(d_1)S - \Phi(d_2)Ke^{-rt})}{\partial t}$

$= S \phi(d_1) \frac{\partial d_1}{\partial t} + rKe^{-rT}\Phi(d_2) -\phi(d_2) \frac{\partial d_2}{\partial t}Ke^{-rT}$


$= -\frac{S \phi(d_1)\sigma}{2 \sqrt{t}} - rK e^{-rT} \Phi(d_2)$


$\Theta_{Call} = -\frac{S \phi(d_1)\sigma}{2 \sqrt{t}} + rK e^{-rT} \Phi(-d_2)$

Important note: It is more meaningful to show the change in value for a day and not for a year. So I use $\Theta' = \frac{1}{365} \cdot \Theta$

In [37]:
def theta_black_scholes(r, S, K, T, sigma, type):

    d1 = (np.log(S/K) + (r+ sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)

    if type == "CALL":
        return ((-S*norm.pdf(d1,0,1)*sigma)/(2*np.sqrt(T)) - r*K*np.exp(-r*T)*norm.cdf(d2) )/365
    elif type == "PUT":
        return ((-S*norm.pdf(d1,0,1)*sigma)/(2*np.sqrt(T)) + r*K*np.exp(-r*T)*norm.cdf(-d2) )/365

type = "CALL"
print("our call option theta: " + str(theta_black_scholes(r, S, K, T, sigma, type)))
print("correct call option theta: " + str(theta("c", S, K, T, r, sigma)))


type = "PUT"
print("our put option theta: " + str(theta_black_scholes(r, S, K, T, sigma, type)))
print("correct put option theta: " + str(theta("p", S, K, T, r, sigma)))

our call option theta: -0.003663899299916886
correct call option theta: -0.003663899299916886
our put option theta: -0.0025751911050726785
correct put option theta: -0.0025751911050726785


# Rho $\rho$

measures the sensitivity to a change in interest rates: $\rho = \frac{\partial V}{\partial r}$

$\rho_{Call}= \frac{\partial (\Phi(d_1)S - \Phi(d_2)Ke^{-rt})}{\partial r}$

$= Kte^{-rT}\Phi(d2)$

$\rho_{Put}= -Kte^{-rT}\Phi(d2)$

Important note: It is more meaningful to report the change in value for a 1% change in interest rate $r$. So I use $\rho' = 0.01 \cdot \rho$

In [34]:
def rho_black_scholes(r, S, K, T, sigma, type):

    d1 = (np.log(S/K) + (r+ sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)

    if type == "CALL":
        return 0.01*K*T*np.exp(-r*T)*norm.cdf(d2)
    elif type == "PUT":
        return 0.01*-K*T*np.exp(-r*T)*norm.cdf(-d2)

type = "CALL"
print("our call option rho: " + str(rho_black_scholes(r, S, K, T, sigma, type)))
print("correct call option rho: " + str(rho("c", S, K, T, r, sigma)))


type = "PUT"
print("our put option rho: " + str(rho_black_scholes(r, S, K, T, sigma, type)))
print("correct put option rho: " + str(rho("p", S, K, T, r, sigma)))

our call option rho: 0.026329642623281514
correct call option rho: 0.026329642623281493
our put option rho: -0.23496032413932819
correct put option rho: -0.23496032413932816
